# Teil 1: Daten laden, Zusammenführen und Bereinigen
## Mit Polars DataFrames

In diesem Notebook werden die Daten geladen, bereinigt und visualisiert. Dabei werden die Polars und Seaborn Bibliotheken verwendet.

Polars bietet uns Datentypen um mit tabellarischen Daten zu arbeiten (DataFrame und Series). Außerdem werden eine Menge von Methoden bereitgestellt um Daten zu bereinigen und zu transformieren. Polars ist eine sehr performante Bibliothek, die auf Apache Arrow basiert und somit sehr gut mit dem Apache Big Data Ökosystem kompatibel ist.  
Sie ist dazu sehr performant und kann, im Gegenzug zur Pandas Bibliothek, sehr große Datenmengen verarbeiten (Out-of-Memory Query Engine).

Numpy und Scipy sind Mathematik bzw. Wissenschaftliche quasi-Standard Bibliotheken und bieten uns eine Vielfalt an bekannten Funktionen und Methoden.

Damit haben wir bereits alles, was wir benötigen, um eine erste Analyse der Daten durchzuführen und mit diesen Ergebnissen später ein Modell zu entwickeln, um eine Vorhersage zu treffen.

In [1503]:
import numpy as np
import scipy as sp
import polars as pl

Zur Veranschaulichung des Analytics/Modeling Prozesses verwenden wir einen kleinen Datensatz der einfach verständlich ist. In der Praxis hat man es oft mit komplexeren Daten zu tun, die eine Menge an Vorverarbeitung und Reduktion benötigen, um sie für das Modellieren zu nutzen.  
Öffentlich verfügbare, reale Datensätze sind leider oft anonymisiert und daher nicht leicht verständlich.

Daten können in unterschiedlichen Formaten vorliegen. Da jedes Unternehmen eine eigene Dateninfrastruktur besitzt, verwenden wir für diesen Workshop Dateien als universelle und menschenlesbare Datenquelle.  

Polars macht es einfach, direkt aus solchen Dateiformaten in einen DataFrame einzulesen.  

An dieser Stelle machen wir uns erst einmal mit dem Datensatz vertraut.  
Die Beschreibung zu diesem synthetischen Datensatz finden Sie unter: https://www.kaggle.com/datasets/lucasokwudishu/gta-v-vehicle-dataset

Warum einen Datensatz aus einem Spiel verwenden?  
Spiele sind eine gute Quelle fuer Daten, da sie eine komplexe Welt simulieren und frei zugänglich sind (Im Gegensatz zu industriellen Daten, die oft anonymisiert sind). Außerdem bestehen sie oft aus stark heterogenen Datentypen, was uns eine gute Gelegenheit bietet, Datenbereinigung und Transformation zu üben.

Hinzu kommt, dass jede/r sich etwas unter den Daten vorstellen kann, was es einfacher macht, die Ergebnisse gemeinsam zu interpretieren.


### Einlesen und Merging der Daten
Wie wir uns eben in der Datenbeschreibung angesehen haben, besteht der Datensatz aus mehreren verschiedenen Dateien. 
Diese müssen wir nun zusammenführen, um einen kompletten Datensatz zu erhalten. Danach werden wir die Daten bereinigen bevor es mit Notebook `02_transform_extract.ipynb` weitergeht.

In [1504]:
# Öffnen sie nebenbei die polars doku https://pola-rs.github.io/polars-book/user-guide/index.html

# try/except ist in einem notebook etwas überflüssig aber es ist eine gute Übung 
# und Vorlage für die spätere Weiterverarbeitung in einer komplexeren Applikation

try:
    df = (pl.read_csv("resources/data/gta_v/gta_data_batch_1.csv", columns=range(1,35), sep=',')
        .extend(pl.read_csv("resources/data/gta_v/gta_data_batch_2.csv", columns=range(1,35), sep=','))
        .extend(pl.read_csv("resources/data/gta_v/gta_data_batch_3.csv", columns=range(1,35), sep=','))
    )
except (IOError, OSError) as e:
    print(f'Failed to read files {e}')
except OtherError as e:
    print(f'Other error {e}')


# check if there are duplicates
if df.shape != df.unique().shape:
    print(f'Warning: {df} has duplicates -> drop them manually')


Ein `print()` des DataFrame liefert einen gekürzten Auszug der Header und des Inhalts und die Dimensionen der Tabelle (shape: Zeilen, Spalten).  
Da wir die harte Arbeit aber dem Computer überlassen wollen, sollte dies nur als Gedankenstütze dienen.  
Weitere Methoden sind `.head(n)` und `.tail(n)` um die ersten bzw. letzten n Zeilen des DataFrame anzuzeigen.  

Mit der Methode `.describe()` kann eine statistische Übersicht über die Daten gegeben werden.

In [1505]:
print(df)

shape: (567, 34)
┌───────────┬────────────┬────────────┬───────────┬─────┬─────────┬──────────┬─────────┬───────────┐
│ title     ┆ vehicle_cl ┆ manufactur ┆ features  ┆ ... ┆ braking ┆ handling ┆ overall ┆ vehicle_u │
│ ---       ┆ ass        ┆ er         ┆ ---       ┆     ┆ ---     ┆ ---      ┆ ---     ┆ rl        │
│ str       ┆ ---        ┆ ---        ┆ str       ┆     ┆ str     ┆ str      ┆ str     ┆ ---       │
│           ┆ str        ┆ str        ┆           ┆     ┆         ┆          ┆         ┆ str       │
╞═══════════╪════════════╪════════════╪═══════════╪═════╪═════════╪══════════╪═════════╪═══════════╡
│ GTA 5:    ┆ Planes     ┆ NA         ┆ Armored   ┆ ... ┆ Braking ┆ Handling ┆ Overall ┆ https://w │
│ Volatol   ┆            ┆            ┆ Vehicle,  ┆     ┆ 32.50   ┆ 1.01     ┆ 34.54   ┆ ww.gtabas │
│           ┆            ┆            ┆ Bombs     ┆     ┆         ┆          ┆         ┆ e.com//gr │
│           ┆            ┆            ┆ Mod,      ┆     ┆         ┆       

Allerdings sind wir noch nicht fertig - Es fehlen noch die Daten der Upgrade-Kosten.  
Diese sind in in zwei Batches inseparaten Dateien gespeichert. Sie enthalten ebenfalls eine Spalte mit der Fahrzeug URL, die wir nutzen können, um die Daten zu verknüpfen (mittels `join`).

In [1506]:
try:
    dfcost = (pl.read_csv("resources/data/gta_v/gta_data_upgrade_cost_1.csv", columns=range(1,3))
        .extend(pl.read_csv("resources/data/gta_v/gta_data_upgrade_cost_2.csv", columns=range(1,3)))
    )
except (IOError, OSError) as e:
    print(f'Failed to read files {e}')
except OtherError as e:
    print(f'Other error {e}')


if dfcost.shape != dfcost.unique().shape:
    print(f'Warning: {dfcost} has duplicates -> drop them manually')

In [1507]:
df = df.join(dfcost, on='vehicle_url', how='left')
del dfcost

Jetzt ist ein wichtiger Aspekt, ob invalide Datenpunkte vorhanden sind. Das können z.b. fehlende Werte sein (Sensorausfall), aber auch ungültige Werte, die nicht in den erwarteten Bereich fallen (Nicht abgefangene Eingaben). 
Typischerweise werden diese in Polars als `null` dargestellt (Pandas: `NA`) und machen den gesamten Datenpunkt unbrauchbar. Numerisch nicht verwendbare Werte wie `+/-inf` oder `nan` sind ebenfalls ein Problem (allerdings sind dies keine _fehlenden_ Werte).

Oft wird man diese Werte einfach aus dem Datensatz entfernen, um das Ergebnis nicht zuverfälschen.
Allerdings kann dies zu einem Verlust von Information führen, wenn die Anzahl der ungültigen Werte zu groß ist.
Daher _kann_ es sinnvoll sein, diese Werte mit speziellen Werten zu ersetzen.
Eine weitere Möglichkeit ist "Clipping" - also das Setzen von Grenzwerten, die außerhalb des erwarteten Bereichs liegen.

In der Praxis ist dies nahezu immer erforderlich, da "echte" Daten häufig "verunreinigt" sind.

Mit folgendem Code können wir die Anzahl der `null` Werte pro Spalte ausgeben lassen, sofern überhaupt welche existieren.

In [1508]:
for col in df.get_columns():
    if col.is_null().sum() > 0:
        print(f'{col.name : <24} {col.is_null().sum()}')

features                 47


In diesem Beispiel sehen wir, dass nur die Spalte `features` null-Werte enthält. 
Allerdings müssen wir hier genau hinschauen, da die Spalte `features` selbst eine Liste von Text-Werten ist.
> Achtung! `null` heißt hier also nur, dass keine besonderen features vorhanden sind - es wäre fatal diese Zeilen zu entfernen!  

Sonst gibt es einen einfachen Weg, alle Zeilen mit mindestens einem `null`-Wert zu entfernen bzw. zu ersetzen: `.drop_nulls()` / `.fill_null()`.

Hier hätten wir dann allerdings bei von 0 und 1 features die gleiche Länge an Listenelementen - also belassen wir es vorerst bei nulls
und erstellen später eine neue Spalte in der wir die Anzahl der Features berechnen.

In [1509]:
print(df.describe())

shape: (7, 36)
┌──────────┬─────────┬────────────┬────────────┬─────┬──────────┬─────────┬───────────┬────────────┐
│ describe ┆ title   ┆ vehicle_cl ┆ manufactur ┆ ... ┆ handling ┆ overall ┆ vehicle_u ┆ upgrade_co │
│ ---      ┆ ---     ┆ ass        ┆ er         ┆     ┆ ---      ┆ ---     ┆ rl        ┆ st         │
│ str      ┆ str     ┆ ---        ┆ ---        ┆     ┆ str      ┆ str     ┆ ---       ┆ ---        │
│          ┆         ┆ str        ┆ str        ┆     ┆          ┆         ┆ str       ┆ str        │
╞══════════╪═════════╪════════════╪════════════╪═════╪══════════╪═════════╪═══════════╪════════════╡
│ count    ┆ 567     ┆ 567        ┆ 567        ┆ ... ┆ 567      ┆ 567     ┆ 567       ┆ 567        │
│ null_cou ┆ 0       ┆ 0          ┆ 0          ┆ ... ┆ 0        ┆ 0       ┆ 0         ┆ 0          │
│ nt       ┆         ┆            ┆            ┆     ┆          ┆         ┆           ┆            │
│ mean     ┆ null    ┆ null       ┆ null       ┆ ... ┆ null     ┆ null    ┆ 

Dies ist eine gute Möglichkeit um einen ersten Eindruck über die Daten zu bekommen.
Zum Beispiel kann das Verhältnis von `mean` zu `std` eine erste Einschätzung über die Präsenz von Außenseitern geben.
Auch ist es wichtig, um die Skalen der Daten zu verstehen - dies wird beim Modellieren wichtig.

### Datenbereinigung / Vorverarbeitung
Nachdem die Daten nun korrekt zusammengeführt wurden, können wir uns um die Datenbereinigung kümmern. 
Konkret fallen bei unserem Datensatz folgende Dinge auf:

1. In Spalte `title` ist ein redundantes "GTA 5:" enthalten
2. In Textspalten sollte vor- und nachstehender Whitespace entfernt werden.
3. Spalte `resale_price` enthält eigentlich 2 separate Werte, die auch in separaten Spalten stehen sollten z.B. `resale_price_base` und `resale_price_upgrade`
4. `top_speed_real`, `top_speed_in_game` enthält mph und kmh. Da wir nicht im letzten Jahrhundert leben, können wir mph verwerfen.
5. `speed`, `acceleration`, `braking`, `handling`, `overall`: Die numerischen Werte sollen extrahiert und die Spalten in den entsprechenden Datenzp umgewandelt werden
6. `upgrade_cost`: Diese Spalten enthalten gemischten Text und numerische Werte. Wir müssen also die numerischen Werte extrahieren und die Spalten in numerische Werte umwandeln und entscheiden, was mit Sonderfällen wie "N/A" passiert.

In [1510]:
# 1. Strip "GTA 5:" from title
# denken Sie an die Zuweisung an df, sonst ändert sich der DataFrame nicht
df = df.with_columns(
    pl.col('title').str.slice(7, None)
)

# 2. Strip leading/trailing whitespace from all text columns
# https://pola-rs.github.io/polars-book/user-guide/howcani/selecting_data/selecting_data_expressions.html
# filter        : select rows
# select        : select columns only return those
# with_columns  : operate on subset of columns but return all
df = df.with_columns(
    pl.col(pl.Utf8).str.strip(" \t\n\r\b\f\v\a")
)

# 3. Split resale_price, then take care of NA/nulls (numeric conversion is handled below)
df = df.with_columns(
    pl.col('resale_price').str.replace('NA', '0(0'), # using this trick for consistent splitting
)
df = df.with_columns(
    (
    pl.col('resale_price')
    .str.split_exact(by='(', n=1)
    .struct.rename_fields(['resale_price_base','resale_price_upgrade'])
    .alias('fields')
    ),
).unnest('fields').drop(['resale_price'])

# 4. discard mph values from top_speed_in_game and top_speed_real because we're not savages
df = df.with_columns(
    (
    pl.col('top_speed_real')
    .str.split_exact(by='(', n=1)
    .struct.rename_fields(['mph','top_speed_real_kmh'])
    .alias('fields')
    ),
).unnest('fields').drop(['mph','top_speed_real'])

df = df.with_columns(
    (
    pl.col('top_speed_in_game')
    .str.split_exact(by='(', n=1)
    .struct.rename_fields(['mph','top_speed_in_game_kmh'])
    .alias('fields')
    ),
).unnest('fields').drop(['mph', 'top_speed_in_game'])

# 5. Strip non-numeric characters from numeric columns resale_price_base and resale_price_upgrade
# this is super ugly and could be done in a more elegant way by just retaining for numeric characters
stripchars = ' $abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ/) \n\t'
numeric_cols = ['resale_price_base', 
                'resale_price_upgrade', 
                'price', 
                'weight_in_kg', 
                'top_speed_real_kmh', 
                'top_speed_in_game_kmh',
                'speed',
                'acceleration',
                'braking',
                'handling',
                'overall',
                'upgrade_cost']
                

df = df.with_columns(
    pl.col(numeric_cols).str.strip(stripchars),
)
df = df.with_columns(
    pl.col(numeric_cols).str.replace_all(r',', ''),
)

# FIXME where $0 in upgrade_cost stripped to "", replace them with "0"
df = df.with_columns(pl.col('upgrade_cost').str.replace('','0'))

df = df.with_columns(
    pl.col(numeric_cols).cast(pl.Float64),
)

#print final data frame
print(df)

shape: (567, 36)
┌───────┬────────────┬────────────┬──────┬─────┬────────────┬────────────┬────────────┬────────────┐
│ title ┆ vehicle_cl ┆ manufactur ┆ feat ┆ ... ┆ resale_pri ┆ resale_pri ┆ top_speed_ ┆ top_speed_ │
│ ---   ┆ ass        ┆ er         ┆ ures ┆     ┆ ce_base    ┆ ce_upgrade ┆ real_kmh   ┆ in_game_km │
│ str   ┆ ---        ┆ ---        ┆ ---  ┆     ┆ ---        ┆ ---        ┆ ---        ┆ h          │
│       ┆ str        ┆ str        ┆ str  ┆     ┆ f64        ┆ f64        ┆ f64        ┆ ---        │
│       ┆            ┆            ┆      ┆     ┆            ┆            ┆            ┆ f64        │
╞═══════╪════════════╪════════════╪══════╪═════╪════════════╪════════════╪════════════╪════════════╡
│ Volat ┆ Planes     ┆ NA         ┆ Armo ┆ ... ┆ 2.2344e6   ┆ 2.421475e6 ┆ 266.35     ┆ 250.0      │
│ ol    ┆            ┆            ┆ red  ┆     ┆            ┆            ┆            ┆            │
│       ┆            ┆            ┆ Vehi ┆     ┆            ┆            ┆

Jetzt haben wir (endlich) einen sauberen Datensatz mit den korrekten Datentypen.

Zum Schluss dieses Arbeitsschritts schreiben wir diesen Datensatz in eine neue Datei, damit wir ihn später einfacher wiederverwenden können.  
Wir verwenden hier das `parquet` Format, welches sehr performant ist und die Daten komprimiert speichert.
CSV ist natuerlich auch eine Möglichkeit, wenn man die Daten weiterhin in einem menschenlesbaren Format haben möchte.  

Bei so einem kleinen Datensatz machen wir einfach beides :)

In [1511]:
df.write_csv("resources/data/gta_v/gta_v_data.csv")
# we can also use a non-human readable (but smaller and faster to load) format, e.g. parquet
df.write_parquet("resources/data/gta_v/gta_v_data.parquet")